In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
crisis1 = pd.read_excel('../data/crisis1.xlsx', skiprows = 2)

In [ ]:
print(crisis1.head())

In [ ]:
crisis2 = pd.read_excel('../data/crisis2.xlsx', skiprows = 2)

In [ ]:
print(crisis2.head())

In [ ]:
crisis1.tail()

In [ ]:
crisis2.tail()

In [ ]:
crisis1.dropna(axis=1, how='all', inplace=True)
crisis1.head()

In [ ]:
crisis2.dropna(axis=1, how='all', inplace=True)
crisis2.head()

In [ ]:
crisis1.info()

In [ ]:
crisis2.info()

In [ ]:
print(crisis1.dtypes)

In [ ]:
print(crisis2.dtypes)

In [ ]:
#I think the best way to read the call volume xlsx is to make individual DataFrames for each section of each tab
#example: 2020_stats as A11:B20, 2020_queue_dist as A24:I36, etc.
#this will require quite a bit of pd.read_excel kwargs but I think it will be worth it (and maybe necessary)

In [ ]:
stats_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=10, nrows=9, usecols='A:B')
stats_2020

In [ ]:
stats_2020.rename(inplace=True, columns={'Total Calls':'Category', 'Unnamed: 1':'Total'})
#stats_2020['Total'].sub(' calls', '') this didn't work the way I wanted it to, commenting it out until I figure out how to fix it
stats_2020

In [ ]:
queue_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=23, nrows=12, usecols='A:I')
queue_dist_2020

In [ ]:
month_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=39, nrows=12, usecols='A:K')
month_dist_2020

In [ ]:
day_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=55, nrows=7, usecols='A:K')
day_dist_2020

In [ ]:
queue_answered_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=66, nrows=10, usecols='A:C')
queue_answered_2020

In [ ]:
service_speed_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=80, nrows=10, usecols='A:D')
service_speed_2020

In [ ]:
abandon_rate_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=94, nrows=10, usecols='A:D')
abandon_rate_2020

##### 2020:
- stats_2020  -- high-level statistics about total number of calls, unanswered/disconnected rates, etc.
- queue_dist_2020 -- distribution of calls across all queue services
- month_dist_2020 -- distribution of calls by month
- day_dist_2020 -- distribution of calls by day of week
- queue_answered_2020 -- answered calls across all queue services
- service_speed_2020 -- speed of answer in time categories
- abandon_rate_2020 -- speed of caller abandoning call in time categories

##### 2021: